In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_labels = path.join(root_folder,'data/train_labels.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 25;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)




In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/3283768 (0%)]	Loss: 0.65695887804031372070
Train Epoch: 0 [450000/3283768 (14%)]	Loss: 0.21818359196186065674
Train Epoch: 0 [900000/3283768 (27%)]	Loss: 0.15728344023227691650
Train Epoch: 0 [1350000/3283768 (41%)]	Loss: 0.13158376514911651611
Train Epoch: 0 [1800000/3283768 (55%)]	Loss: 0.12426821142435073853
Train Epoch: 0 [2250000/3283768 (68%)]	Loss: 0.11324431002140045166
Train Epoch: 0 [2700000/3283768 (82%)]	Loss: 0.11344414204359054565
Train Epoch: 0 [3150000/3283768 (96%)]	Loss: 0.10949877649545669556

Test set: Average loss: 0.00000651789105177158, Accuracy: 793311/820942 (96.63423238182478769431%)

Train Epoch: 1 [0/3283768 (0%)]	Loss: 0.10637089610099792480
Train Epoch: 1 [450000/3283768 (14%)]	Loss: 0.09994456917047500610
Train Epoch: 1 [900000/3283768 (27%)]	Loss: 0.09409442543983459473
Train Epoch: 1 [1350000/3283768 (41%)]	Loss: 0.09631362557411193848
Train Epoch: 1 [1800000/3283768 (55%)]	Loss: 0.09567553550004959106
Train Epoch: 1 [2250000/3283768 (

Train Epoch: 13 [0/3283768 (0%)]	Loss: 0.06808773428201675415
Train Epoch: 13 [450000/3283768 (14%)]	Loss: 0.06962320953607559204
Train Epoch: 13 [900000/3283768 (27%)]	Loss: 0.06674188375473022461
Train Epoch: 13 [1350000/3283768 (41%)]	Loss: 0.06589924544095993042
Train Epoch: 13 [1800000/3283768 (55%)]	Loss: 0.06860180199146270752
Train Epoch: 13 [2250000/3283768 (68%)]	Loss: 0.06786905974149703979
Train Epoch: 13 [2700000/3283768 (82%)]	Loss: 0.06779435276985168457
Train Epoch: 13 [3150000/3283768 (96%)]	Loss: 0.07000218331813812256

Test set: Average loss: 0.00000416155216953484, Accuracy: 802646/820942 (97.77134072809040787888%)

Train Epoch: 14 [0/3283768 (0%)]	Loss: 0.07151057571172714233
Train Epoch: 14 [450000/3283768 (14%)]	Loss: 0.06304610520601272583
Train Epoch: 14 [900000/3283768 (27%)]	Loss: 0.06765827536582946777
Train Epoch: 14 [1350000/3283768 (41%)]	Loss: 0.07225447148084640503
Train Epoch: 14 [1800000/3283768 (55%)]	Loss: 0.06459092348814010620
Train Epoch: 14 [225

In [7]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00000415953354604426, Accuracy: 802656/820942 (97.77255884094125804040%)

